In [ ]:
!pip install datasets
!pip install transformers
# from google.colab import files
# uploaded = files.upload()

In [ ]:
print("HI folks")

HI folks


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
train_file_path = "/content/drive/My Drive/Academic/Courses/NLP.Fa22/Assignments/hw4/train and test files/redditComments_train.jsonlist"
test_file_path = "/content/drive/My Drive/Academic/Courses/NLP.Fa22/Assignments/hw4/train and test files/redditComments_test_notGraded.jsonlist"

Mounted at /content/drive


In [ ]:
from datasets import Dataset, load_dataset, load_metric
import numpy as np
import pandas as pd
import tensorflow as tf
import torch
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification, TrainingArguments
import os
import json

# dataset = load_dataset("json",
	# data_files={'train':"train.jsonlist", 'test':"test_nonGraded.jsonlist"})

In [ ]:
#dataset['test']['parent_comment'] + '[SEP]' + dataset['test']['parent_comment']
import random
import re
name_to_num = {}
def json_to_pandas(path):
  dataset = []
  current_label_num = -1
  with open(path) as F:
    for l in F.readlines():
      l = json.loads(l)
      newL = dict()
      if l['subreddit'] not in name_to_num.keys():
            current_label_num += 1
            name_to_num[l['subreddit']] = current_label_num
      newL['labels'] = name_to_num[l['subreddit']]
      l['body'] = re.sub(r'[^A-Za-z. ]', ' ', l['body'])
      newL['text'] = l['body']
      dataset.append(newL)
  random.shuffle(dataset)

  #l = len(dataset)
  #n = int(1 * l / 10 - 1)
  #dataset = dataset[0:n]

  dataset = pd.DataFrame.from_dict(dataset)
  return Dataset.from_pandas(dataset)

dataset_train = json_to_pandas(train_file_path)
dataset_test = json_to_pandas(test_file_path)

print(dataset_train)
print(dataset_test)

Dataset({
    features: ['labels', 'text'],
    num_rows: 19600
})
Dataset({
    features: ['labels', 'text'],
    num_rows: 200
})


In [ ]:
#dataset['test']['parent_comment'] + '[SEP]' + dataset['test']['parent_comment']
def without_top_words_json_to_pandas(path,top_list):
  dataset = []
  current_label_num = -1
  with open(path) as F:
    for l in F.readlines():
      l = json.loads(l)
      newL = dict()
      newL['labels'] = name_to_num[l['subreddit']]
      l['body'] = re.sub(r'[^A-Za-z. ]', '', l['body'])
      for word in top_list:
        l['body'] = l['body'].lower()
        if word == '.' or word == ')' or word == '(' or word == '[' or word == ']' or word == '{' or word == '}':
          word = '\\' + word
        else:
          word = '(^|[^a-zA-Z])' + word + '([^a-zA-Z]|$)'
        l['body']= re.sub(r''+ word, ' ',  l['body'])
      newL['text'] = l['body']
      dataset.append(newL)
  random.shuffle(dataset)

  #l = len(dataset)
  #n = int(1 * l / 10 - 1)
  #dataset = dataset[0:n]

  dataset = pd.DataFrame.from_dict(dataset)
  return Dataset.from_pandas(dataset)


In [ ]:
print("building tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def tokenize_function(examples):
	return tokenizer(examples["text"], padding="max_length", truncation=True)
tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)

n_max_key_value = None
n_max_key = []
def top_n_scores(n,dataset):
  ''' returns a list of the most frequent values of a list '''
  frequency = {}
  for i in range (len(dataset)):
    for num in dataset[i]['input_ids']:
      if num not in frequency.keys():
        frequency[num] = 1
      else:
        frequency[num] += 1
  l = int(len(frequency) * n / 100)
  lot = [(k,v) for k, v in frequency.items()] #make list of tuple from scores dict
  max_key = []
  while len(lot)> 0:
      max_key.append(max(lot, key=lambda x: x[1]))
      lot.remove(max_key[-1])
  n_max_key_value = max_key[0:l]
  #print("n_max_key_value", n_max_key_value)
  for key,value in n_max_key_value:
    n_max_key.append(key)
  return n_max_key
'''这里 '''
n_max_key_train = top_n_scores(0.5,tokenized_dataset_train)
print(n_max_key_train)
top_list = [tokenizer.decode(i) for i in n_max_key_train]
print(top_list)

print('Removing high frequency words.')
dataset_train = without_top_words_json_to_pandas(train_file_path,top_list)
dataset_test = without_top_words_json_to_pandas(test_file_path,top_list)
print('high frequency words removed')
print("building tokenizer without top words...")
tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)
n_max_key_value = None
n_max_key = []
n_max_key_train = top_n_scores(0.5,tokenized_dataset_train)
print(n_max_key_train)
top_list = [tokenizer.decode(i) for i in n_max_key_train]
print(top_list)

building tokenizer...


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

  0%|          | 0/1 [00:00<?, ?ba/s]

[0, 1012, 1996, 1045, 101, 102, 1037, 2000, 2017, 1998, 2009, 1997, 2008, 2003, 1999, 2005, 1055, 1056, 2021, 2006, 2031, 2007, 2023, 2026, 2065, 2022, 2115, 2504, 2001, 2004, 2064, 2061, 2024, 2074, 2015, 2025, 2066, 2027, 2012, 2131, 2030, 2028, 2033, 2079, 1049, 1040, 2054, 2041, 2039, 2097, 2045, 2128, 2019, 2123, 2428, 2204, 2035, 2052, 2062, 2013, 2070, 2866, 2377, 2222, 2121, 2051, 2055, 2043, 2191, 2068, 2053, 2036]
['[PAD]', '.', 'the', 'i', '[CLS]', '[SEP]', 'a', 'to', 'you', 'and', 'it', 'of', 'that', 'is', 'in', 'for', 's', 't', 'but', 'on', 'have', 'with', 'this', 'my', 'if', 'be', 'your', 'level', 'was', 'as', 'can', 'so', 'are', 'just', '##s', 'not', 'like', 'they', 'at', 'get', 'or', 'one', 'me', 'do', 'm', 'd', 'what', 'out', 'up', 'will', 'there', 're', 'an', 'don', 'really', 'good', 'all', 'would', 'more', 'from', 'some', 'uk', 'play', 'll', '##er', 'time', 'about', 'when', 'make', 'them', 'no', 'also']
Removing high frequency words.
high frequency words removed
buil

  0%|          | 0/20 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

[0, 101, 102, 2102, 2015, 2049, 2063, 10047, 2860, 2123, 2121, 2866, 2228, 2115, 2129, 4283, 2113, 2175, 2034, 2205, 2075, 2038, 2215, 2092, 2200, 2011, 2094, 2172, 2018, 2017, 2138, 2288, 2008, 2059, 2307, 2151, 8909, 2069, 2084, 4569, 2126, 2571, 3046, 8479, 2060, 2085, 4921, 3835, 2071, 2111, 3798, 3363, 2002, 9307, 2224, 2843, 2546, 2290, 4012, 2183, 2147, 2607, 2652, 2182, 2157, 2156, 2029, 2130, 2469, 2046, 2044, 2145, 5665, 2342]
['[PAD]', '[CLS]', '[SEP]', '##t', '##s', 'its', '##e', 'im', '##w', 'don', '##er', 'uk', 'think', 'your', 'how', 'thanks', 'know', 'go', 'first', 'too', '##ing', 'has', 'want', 'well', 'very', 'by', '##d', 'much', 'had', 'you', 'because', 'got', 'that', 'then', 'great', 'any', 'id', 'only', 'than', 'fun', 'way', '##le', 'try', 'blast', 'other', 'now', 'iv', 'nice', 'could', 'people', 'levels', '##ll', 'he', '##ule', 'use', 'lot', '##f', '##g', 'com', 'going', 'work', 'course', 'playing', 'here', 'right', 'see', 'which', 'even', 'sure', 'into', 'after',

In [ ]:
print(tokenized_dataset_train[0])
input_ids=[]
attention_masks=[]
labels=[]
for i in range(len(tokenized_dataset_train)):
  input_ids.append(tokenized_dataset_train[i]['input_ids'])
  attention_masks.append(tokenized_dataset_train[i]['attention_mask'])
  labels.append(tokenized_dataset_train[i]['labels'])
print(len(input_ids))

input_ids = np.asarray(input_ids)
attention_masks = np.array(attention_masks)
labels = np.array(labels)

train_inp,val_inp,train_label,val_label,train_mask,val_mask=train_test_split(input_ids,labels,attention_masks,test_size=0.2)
print('Train inp shape {} Val input shape {}\nTrain label shape {} Val label shape {}\nTrain attention mask shape {} Val attention mask shape {}'.format(train_inp.shape,val_inp.shape,train_label.shape,val_label.shape,train_mask.shape,val_mask.shape))

{'labels': 2, 'text': ' everyone else had said correct rdcs sitting fish bowl keeping eye things youre lucky theyll even decent music everyone listen  thats only we heard something other than cadences people used iron their inspection items throughly clean their bunks study ', 'input_ids': [101, 3071, 2842, 2018, 2056, 6149, 16428, 6169, 3564, 3869, 4605, 4363, 3239, 2477, 2115, 2063, 5341, 2027, 3363, 2130, 11519, 2189, 3071, 4952, 2008, 2015, 2069, 2057, 2657, 2242, 2060, 2084, 23620, 2015, 2111, 2109, 3707, 2037, 10569, 5167, 2083, 2135, 4550, 2037, 25277, 2015, 2817, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [ ]:
model = TFAutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 4)
#model.cuda()
''' the output path is here! '''
model_save_path="/content/drive/My Drive/Academic/Courses/NLP.Fa22/Assignments/hw4/model out/bestModel"
callbacks = [tf.keras.callbacks.ModelCheckpoint(filepath=model_save_path,
                                                save_weights_only=True,
																								monitor='val_accuracy',
																								verbose=1,
																								mode='max',
																								save_best_only=True),
													tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', min_delta=.01, patience=4)]

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-7)
model.compile(loss=loss,optimizer=optimizer, metrics=[metric])

history=model.fit([train_inp,train_mask],train_label,batch_size=8,epochs=10,validation_data=([val_inp,val_mask],val_label),callbacks=callbacks)

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'vocab_layer_norm', 'vocab_transform', 'activation_13']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier', 'dropout_19', 'pre_classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

Epoch 1/10
1960/1960 [==============================] - ETA: 0s - loss: 1.2326 - accuracy: 0.5430
Epoch 1: val_accuracy improved from -inf to 0.71735, saving model to /content/drive/My Drive/Academic/Courses/NLP.Fa22/Assignments/hw4/model out/bestModel
1960/1960 [==============================] - 138s 66ms/step - loss: 1.2326 - accuracy: 0.5430 - val_loss: 0.9680 - val_accuracy: 0.7173
Epoch 2/10
1960/1960 [==============================] - ETA: 0s - loss: 0.8255 - accuracy: 0.7340
Epoch 2: val_accuracy improved from 0.71735 to 0.74847, saving model to /content/drive/My Drive/Academic/Courses/NLP.Fa22/Assignments/hw4/model out/bestModel
1960/1960 [==============================] - 128s 65ms/step - loss: 0.8255 - accuracy: 0.7340 - val_loss: 0.7134 - val_accuracy: 0.7485
Epoch 3/10
1960/1960 [==============================] - ETA: 0s - loss: 0.6702 - accuracy: 0.7621
Epoch 3: val_accuracy improved from 0.74847 to 0.76990, saving model to /content/drive/My Drive/Academic/Courses/NLP.Fa22

In [ ]:
from sklearn.metrics import precision_score
test_inp=[]
test_mask=[]
test_label=[]
for i in range(len(tokenized_dataset_test)):
  test_inp.append(tokenized_dataset_test[i]['input_ids'])
  test_mask.append(tokenized_dataset_test[i]['attention_mask'])
  test_label.append(tokenized_dataset_test[i]['labels'])
print(len(test_inp))

test_inp = np.asarray(test_inp)
test_mask = np.array(test_mask)
test_label = np.array(test_label)

model.load_weights(model_save_path)
predict = model.evaluate([test_inp,test_mask],test_label)


200
7/7 [==============================] - 2s 66ms/step - loss: 0.4076 - accuracy: 0.8550
